# Solution for Day 8: Treetop Tree House

*Advent of Code 2022*

- [Day 8 Challenge](https://adventofcode.com/2022/day/8)

- [Instructions](instructions.md)

- [Input Data](input.txt)

---

In [1]:
# imports
import numpy as np

In [2]:
INPUT_FILE = 'input.txt'

In [3]:
with open(INPUT_FILE, 'r') as f:
    lines = [line.strip('\n') for line in f.readlines()]
    
# lines

---

## Part I

### Create Rows and Columns from Input Data

In [4]:
# Change lines to a list of lists where each list contains numbers

def create_array(lines):
    """Create an array from a list of numeric strings."""
    rows = [[int(num) for num in list(line)] for line in lines]
    rows = np.array(rows)
    return rows

In [5]:
# Create rows of numbers from the input data
rows = create_array(lines)

# Check that the number of rows and columns in the columns array are equal
# rows.shape

In [6]:
# Now we can transpose the rows to "look down" columns as well
columns = np.transpose(rows)

# Check that the number of rows and columns in the columns array are equal
# columns.shape

### Count Trees on Outer Edges

In [7]:
def count_visible_edge_trees(rows, cols):
    """Counts the number of trees on the edges of the array."""
    
    # Ensure that the number of items in rows is the same as in columns.
    if len(rows[0]) == len(cols[0]):
        # The number of trees on the edge 
        outer_visible = (len(rows[0]) - 1) * 4
        
        return outer_visible
    
    # If rows and columns are not a true array, return an error message
    else:
        return f'Error: Length of rows ({len(rows)}) does not match length of columns ({len(columns)})'


In [8]:
count_visible_edge_trees(rows, columns)

392

---

## Directional Views

### Views for Rows

In [248]:
def split_row(row_of_trees, left_to_right=True):
    """
    Split a row of trees into two parts: the first part is all of the trees
    up to the last occurence of the tallest tree, and the second part is the rest of the 
    trees.
    """
    
    # Find the height of the tallest tree in the row of trees.
    tallest_tree = max(row_of_trees)

    # Find the index of the LAST occurence of the tallest tree in the row (when looking
    # down the row from left to right), because any trees beyond it will not be visible.
    # To do so, make a list of indices where the tallest trees occur in the row, then
    # find the max value.
    last_tallest_tree_index = max([i for i, tree in enumerate(row_of_trees)
                                   if tree == tallest_tree])

    # This is the part of the row we will be working with for the left-to-right view.
    row_of_trees_left_to_right = row_of_trees[:last_tallest_tree_index+1]
    
    # Keep the rest of the row handy for when we need to look right-to-left, as we don't
    # want to accidentally count any trees twice.
    row_of_trees_right_to_left = row_of_trees[last_tallest_tree_index+1:]
    row_of_trees_right_to_left
    
    if left_to_right == True:
        return row_of_trees_left_to_right
    
    elif left_to_right == False:
        return row_of_trees_right_to_left

In [281]:
def look_left_to_right(row_of_trees):
    
    # We can now begin to iterate through our chunk of the row of trees. Each tree in our
    # chunk needs to be shorter than the tree that comes after it to be visible.
    # To start, set the first tree in the row as the "current" tallest tree.
    visible = []
    visible_trees = 0
    
    current_tallest = row_of_trees[0]
    
    # Add this tree to the visible trees list.
    visible.append(current_tallest)

    # Starting from the second tree in the new row, check the height of that tree against
    # the current tallest tree.
    for tree in row_of_trees[1:]:
        # If the tree is taller than the current tallest tree, add that tree to the 
        # list of visible trees.
        if tree > current_tallest:
            visible.append(tree)
            # Then, set the new current tallest tree value to that tree's height.
            current_tallest = tree
        # Otherwise, keep checking the rest of the trees until we have reached the last
        # tree in the new row.
        else:
            continue
        
    visible_trees += len(visible)
    # return visible_trees
    return visible

In [278]:
def look_right_to_left(row_of_trees):
    visible = []
    visible_trees = 0

    # Since we are looking right-to-left, reverse the section of the row of trees we have
    # been given.
    reversed_row_of_trees = row_of_trees[:]
    reversed_row_of_trees.reverse()
    
    # Find the height of the tallest tree in the row of trees.
    tallest_tree = max(row_of_trees)

    # Find the index of the LAST occurence of the tallest tree in the row (when looking
    # down the row from left to right), because any trees beyond it will not be visible.
    # To do so, make a list of indices where the tallest trees occur in the row, then
    # find the max value.
    last_tallest_tree_index = max([i for i, tree in enumerate(reversed_row_of_trees)
                                   if tree == tallest_tree])

    # Keep the rest of the row handy for when we need to look right-to-left, as we don't
    # want to accidentally count any trees twice.
    reversed_row_of_trees_up_to_tallest_tree = reversed_row_of_trees[:last_tallest_tree_index+1]
    
    print(reversed_row_of_trees_up_to_tallest_tree)
    # We can now begin to iterate through our chunk of the row of trees. Each tree in our
    # chunk needs to be shorter than the tree that comes after it to be visible.
    # To start, set the first tree in the row as the "current" tallest tree.
    current_tallest = reversed_row_of_trees_up_to_tallest_tree[0]
    
    # Add this tree to the visible trees list.
    visible.append(current_tallest)

    # Starting from the second tree in the new row, check the height of that tree against
    # the current tallest tree.
    for tree in reversed_row_of_trees_up_to_tallest_tree[1:]:
        # If the tree is taller than the current tallest tree, add that tree to the 
        # list of visible trees.
        if tree > current_tallest:
            visible.append(tree)
            # Then, set the new current tallest tree value to that tree's height.
            current_tallest = tree
        # Otherwise, keep checking the rest of the trees until we have reached the last
        # tree in the new row.
        else:
            continue

    visible_trees += len(visible)
    
    
    # return visible_trees 
    return visible   

---

### Example

In [266]:
row_trees = [1, 2, 3, 2, 3, 4, 3, 5, 1, 6, 3, 5, 2, 1]

In [267]:
row_of_trees_left_to_right = split_row(row_trees, left_to_right=True)
row_of_trees_left_to_right

[1, 2, 3, 2, 3, 4, 3, 5, 1, 6]

In [273]:
row_of_trees_right_to_left = split_row(row_trees, left_to_right=False)
row_of_trees_right_to_left

[3, 5, 2, 1]

In [257]:
assert row_of_trees_left_to_right + row_of_trees_right_to_left == row_trees

In [269]:
total_visible_trees = 0

In [282]:
look_left_to_right(row_of_trees_left_to_right)



[1, 2, 3, 4, 5, 6]

In [284]:
total_visible_trees += look_right_to_left(row_of_trees_right_to_left)

[1, 2, 5]


TypeError: unsupported operand type(s) for +=: 'int' and 'list'

In [ ]:
total_visible_trees += len(look_left_to_right)

### Views for Columns

In [12]:
def look_down(column):
    pass

In [ ]:
def look_up(column):
    pass

---

### Testcases

In [9]:
example_input = """30373
25512
65332
33549
35390
"""

example_lines = [line.strip('\n') for line in example_input.split()]

example_rows = create_array(example_lines)
example_cols = np.transpose(example_rows)

# outer = (len(example_rows[0]) - 1) * (len(example_cols[0] - 1))
# outer

def find_number_edge_trees(rows, cols):
    outer_visible = (len(rows[0]) - 1) * (len(cols[0]) - 1)
    return outer_visible

print(find_number_edge_trees(example_rows, example_cols))

16


In [195]:
# A single row of trees
row_of_trees = [1, 2, 3, 2, 1, 4, 5, 4, 3, 2, 1, 1, 0]


def look_right(row):
    # Find the height of the tallest tree in the row
    tallest_tree = max(row_of_trees)

    # Find the *first* instance of the tallest tree (when looking down the row from
    # left to right, this is the tree that will block any other trees past it.)
    first_tallest_tree = row_of_trees.index(tallest_tree)

    # Create a new row from the beginning of the row to the first tallest tree.
    new_row = row_of_trees[0:first_tallest_tree+1]

    # Empty list to store the visible trees.
    visible = []

    # Start with the first tree in the new row. As of now, it is currently the tallest
    # tree in the new row by which all other trees will be measured, until a tree comes
    # along that is higher than it.
    current_tallest = new_row[0]
    # Add this tree to the visible trees list.
    visible.append(current_tallest)

    # Starting from the second tree in the new row, check the height of that tree against
    # the current tallest tree.
    for tree in new_row[1:]:
        if tree > current_tallest:
            # If the tree is taller than the tallest tree, add that tree to the 
            # list of visible trees.
            visible.append(tree)
            # Then, set the new current tallest tree value to that tree's height.
            current_tallest = tree
        # Otherwise, keep checking the rest of the trees until we have reached the last
        # tree in the new row.
        else:
            continue
    return len(visible)

5

### Answer

---

## Part II

### Answer